# Baseline and Evaluation

## Imports

In [9]:
import elasticsearch
import os
import re
import string
import time
import numpy as np
import json

# stop words
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))


from tqdm import tqdm
from elasticsearch import Elasticsearch, helpers, exceptions
from typing import Dict, Tuple

# path variables, etc.
from config import *

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/azureuser/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [10]:
es = Elasticsearch(hosts=["http://localhost:9200"])
es.info()

/tmp/ipykernel_9796/208943912.py:2: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  es.info()


ObjectApiResponse({'name': 'yohannes-vm', 'cluster_name': 'elasticsearch', 'cluster_uuid': '1HOaRd2USZ6xr57dnRGoig', 'version': {'number': '7.17.7', 'build_flavor': 'default', 'build_type': 'deb', 'build_hash': '78dcaaa8cee33438b91eca7f5c7f56a70fec9e80', 'build_date': '2022-10-17T15:29:54.167373105Z', 'build_snapshot': False, 'lucene_version': '8.11.1', 'minimum_wire_compatibility_version': '6.8.0', 'minimum_index_compatibility_version': '6.0.0-beta1'}, 'tagline': 'You Know, for Search'})

## Helper functions

In [11]:
def preprocess(line, remove_stopwords=False):
    line = line.strip().lower().replace("_", " ").translate(str.maketrans('', '', string.punctuation))
    return " ".join([
        term 
        for term in re.sub(r"\s+", " ", line).split(" ") 
        if term not in stop_words
    ]).strip() if remove_stopwords else line

In [12]:
# read train and test queries
def load_queries(train_query_path: str, test_query_path: str) -> Tuple[Dict]:
    files = [train_query_path, test_query_path]
    train_queries, test_queries = {}, {}

    for file in files:
        print(f"Loading queries from {file}")
        with open(file, "r") as f:
            try:
                data = json.loads(f.read())
                for query in data:
                    query_id, query_question = query.get('id', '').lower(), query.get('question', '')

                    # some of the questions are null, check for that
                    if not query_question: continue

                    if file == train_query_path:
                        query_category, query_type = query.get('category', '').lower(), ' '.join(query.get('type', [])).replace("dbo:", "").lower()
                        if query_category != 'resource': continue #only consider resource queries
                        train_queries[query_id] = {"query": preprocess(query_question.lower()), "category": query_category, "type": query_type}
                    elif file == test_query_path:
                        test_queries[query_id] = {"query": preprocess(query_question.lower())}
            except Exception as e:
                print(e, query)
    return train_queries, test_queries

In [13]:
train_queries, test_queries = load_queries(QUERY_TRAIN_PATH, QUERY_TEST_PATH)
print("Train queries: ", len(train_queries), "Test queries: ", len(test_queries), "\n")

print("Sample Train query: ", train_queries['dbpedia_21160'])
print("Sample Test query: ", test_queries['dbpedia_14677'])

Loading queries from /mntnvme/datasets/DBpedia/smarttask_dbpedia_train.json
Loading queries from /mntnvme/datasets/DBpedia/smarttask_dbpedia_test_questions.json
Train queries:  9573 Test queries:  4369 

Sample Train query:  {'query': 'what are some bands out to texarkana', 'category': 'resource', 'type': 'band group organisation agent'}
Sample Test query:  {'query': 'what is the newspaper with the max publication interval'}


## Baseline

In [14]:
def analyze_query(es: Elasticsearch, query: str, index_name: str = INDEX_NAME):
    """Analyzes a query with respect to the relevant index.
       Ref: Assigment 3 - DAT640

    Args:
        es: Elasticsearch object instance.
        query: String of query terms.
        index_name: Name of the index with respect to which the query is analyzed.

    Returns:
        A list of query terms that exist in abstracts.
    """
    tokens = es.indices.analyze(index=index_name, body={"text": query})["tokens"]
    query_terms = []

    for token in sorted(tokens, key=lambda x: x["position"]):
        hits = es.search(index=index_name,
                         body={'query': {'match': {'abstract': token['token']}}},
                         _source=False, size=1).get('hits', {}).get('hits', {})
        doc_id = hits[0]['_id'] if len(hits) > 0 else None
        if doc_id is None: continue
        query_terms.append(token['token'])
    return query_terms

In [15]:
for query_id, query in tqdm(train_queries.items()): 
    train_queries[query_id]['query_terms'] = analyze_query(es, query['query'])

for query_id, query in tqdm(test_queries.items()):
    test_queries[query_id]['query_terms'] = analyze_query(es, query['query'])

  0%|          | 0/9573 [00:00<?, ?it/s]/tmp/ipykernel_9796/4080730600.py:13: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  tokens = es.indices.analyze(index=index_name, body={"text": query})["tokens"]
/tmp/ipykernel_9796/4080730600.py:13: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  tokens = es.indices.analyze(index=index_name, body={"text": query})["tokens"]
/tmp/ipykernel_9796/4080730600.py:17: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  hits = es.search(index=index_name,
/tmp/ipykernel_9796/4080730600.py:17: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without 

In [16]:
print("Sample Train query: ", train_queries['dbpedia_21160'])

Sample Train query:  {'query': 'what are some bands out to texarkana', 'category': 'resource', 'type': 'band group organisation agent', 'query_terms': ['what', 'some', 'bands', 'out', 'texarkana']}


### BM25

In [19]:
def evaluate_retrieval(es, train_queries, index_name=INDEX_NAME, k=10):
    # evaluate retrieval
    train_retrieval_results = {}
    hit_count = 0
    for query_id, query_values in tqdm(train_queries.items()):
        try:
            hits = es.search(index=index_name, _source=True, 
                    body={"query": 
                            { "bool": {"must": {"match": {"abstract": " ".join(query_values["query_terms"])}},
                                      "must_not": {"match": {"instance_type": "_"}}}
                            }}, size=k)["hits"]["hits"]
            hit_count += len(hits)
            train_retrieval_results[query_id] = [hit['_source']['instance_type'] for hit in hits]
        except Exception as e:
            print(e, query_values)
    
    return train_retrieval_results

In [20]:
train_retrieval_results = evaluate_retrieval(es, train_queries)
print("Train retrieval results: ", len(train_retrieval_results))

  0%|          | 0/9573 [00:00<?, ?it/s]/tmp/ipykernel_9796/3803168678.py:7: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  hits = es.search(index=index_name, _source=True,
/tmp/ipykernel_9796/3803168678.py:7: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  hits = es.search(index=index_name, _source=True,
100%|██████████| 9573/9573 [05:27<00:00, 29.24it/s]

Train retrieval results:  9573


In [25]:
for query_id, results in list(train_retrieval_results.items())[:10]:
    print(query_id, results)

dbpedia_14427 ['thing', 'film', 'film', 'album', 'holiday', 'musical', 'televisionepisode', 'album', 'album']
dbpedia_3681 ['book', 'televisionshow', 'thing', 'film', 'person', 'film', 'person', 'televisionshow', 'film', 'film']
dbpedia_12020 ['thing', 'country', 'administrativeregion', '_', 'thing', 'administrativeregion', 'legislature', '_', '_']
dbpedia_10315 ['_', '_', '_', '_', '_', 'thing', '_', '_', 'thing']
dbpedia_1335 ['person', 'officeholder', '_', '_', 'film', 'officeholder', 'person', 'film', '_']
dbpedia_6016 ['_', '_', '_', '_', '_', '_', 'thing', '_', '_']
dbpedia_3432 ['thing', 'thing', '_', '_', 'film', '_', 'saint', '_', 'thing']
dbpedia_16006 ['officeholder', '_', 'station', 'crater', '_', 'thing', 'person', '_', 'book']
dbpedia_278 ['person', 'person', 'person', 'person', 'person', 'person', 'person', 'person', 'person', 'award']
dbpedia_7661 ['soccerplayer', 'soccerplayer', 'thing', 'soccerplayer', 'soccerplayer', 'thing', 'soccermanager', '_', 'governor']


## Evaluation

**Code from [this](https://github.com/BerntA/IR-SMART) repo to do the evaluation.**

In [34]:
print(DBPEDIA_TYPES_PATH)

/mntnvme/datasets/DBpedia/dbpedia_types.tsv


In [35]:
def loadDBPediaTypes():
    kv = {}
    max_depth = 0
    with open(DBPEDIA_TYPES_PATH, 'r') as f:
        for i, line in enumerate(f):
            if i == 0: # Skip header
                continue
            line = line.strip().lower().split('\t')
            if len(line) != 3:
                continue
            type_name, depth, parent_type = line[0].split(':')[-1], int(line[1]), line[-1].split(':')[-1]
            if (len(type_name) == 0) or (len(parent_type) == 0):
                continue
            kv[type_name] = {'depth':depth, 'parent':parent_type}
            max_depth = max(depth, max_depth)
    return kv, max_depth

type_hierarchy, max_depth = loadDBPediaTypes()

In [36]:
def getTypeHierarchy(kv, items, target):
    if not target in kv:
        return
    items.append(target)
    getTypeHierarchy(kv, items, kv[target]['parent'])

def buildDBPediaTypeHierarchy(kv, target, reverse=True):
    items = [] # List of types, representing the hierarchy of the types related to the target.
    getTypeHierarchy(kv, items, target)
    if reverse:
        return items[::-1] # Reverse the order to return the correct hierarchy where the first item = top level.
    return items

def cacheDBPediaPaths():
    """Simplify Evaluation Path Computations"""
    for k in type_hierarchy.keys():
        type_hierarchy[k]['path'] = buildDBPediaTypeHierarchy(type_hierarchy, k, False)

In [37]:
buildDBPediaTypeHierarchy(type_hierarchy, 'comic') # Example hierarchy

['work', 'writtenwork', 'comic']

In [46]:
import math

def dcg(gains, k=5):
    """
    Computes DCG for a given ranking.
    Traditional DCG formula: DCG_k = sum_{i=1}^k gain_i / log_2(i+1).
    """
    dcg = 0
    for i in range(0, min(k, len(gains))):
        dcg += gains[i] / math.log(i + 2, 2)
    return dcg

def ndcg(gains, ideal_gains, k=5):
    """Computes NDCG given gains for a ranking as well as the ideal gains."""
    try:
        return dcg(gains, k) / dcg(ideal_gains, k)
    except:
        return 0

def get_type_path(type, type_hierarchy):
    """
    Gets the type's path in the hierarchy (excluding the root type, like owl:Thing).
    The path for each type is computed only once then cached in type_hierarchy,
    to save computation.
    """
    if not type in type_hierarchy:
        type_hierarchy[type] = {'depth':1, 'parent':'', 'path':[type]}
    if "path" in type_hierarchy[type]:
        return type_hierarchy[type]['path']
    return []

def get_type_distance(type1, type2, type_hierarchy):
    """
    Computes the distance between two types in the hierarchy.
    Distance is defined to be the number of steps between them in the hierarchy,
    if they lie on the same path (which is 0 if the two types match), and
    infinity otherwise.
    """
    type1_path = get_type_path(type1, type_hierarchy)
    type2_path = get_type_path(type2, type_hierarchy)
    distance = math.inf
    if type1 in type2_path:
        distance = type2_path.index(type1)
    if type2 in type1_path:
        distance = min(type1_path.index(type2), distance)
    return distance

def get_most_specific_types(types, type_hierarchy):
    """Filters a set of input types to most specific types w.r.t the type
    hierarchy; i.e., super-types are removed."""
    filtered_types = set(types)
    for type in types:
        type_path = get_type_path(type, type_hierarchy)
        for supertype in type_path[1:]:
            if supertype in filtered_types:
                filtered_types.remove(supertype)
    return filtered_types

def get_expanded_types(types, type_hierarchy):
    """Expands a set of types with both more specific and more generic types
    (i.e., all super-types and sub-types)."""
    expanded_types = set()
    for type in types:
        # Adding all supertypes.
        expanded_types.update(get_type_path(type, type_hierarchy))
        # Adding all subtypes (NOTE: this bit could be done more efficiently).
        for type2 in type_hierarchy:
            if type_hierarchy[type2]['depth'] <= type_hierarchy[type]['depth']:
                continue
            type2_path = get_type_path(type2, type_hierarchy)
            if type in type2_path:
                expanded_types.update(type2_path)
    return expanded_types

def compute_type_gains(predicted_types, gold_types, type_hierarchy, max_depth):
    """Computes gains for a ranked list of type predictions.

    Following the definition of Linear gain in (Balog and Neumayer, CIKM'12),
    the gain for a given predicted type is 0 if it is not on the same path with
    any of the gold types, and otherwise it's $1-d(t,t_q)/h$ where $d(t,t_q)$ is
    the distance between the predicted type and the closest matching gold type
    in the type hierarchy and h is the maximum depth of the type hierarchy.

    Args:
        predicted_types: Ranked list of predicted types.
        gold_types: List/set of gold types (i.e., perfect answers).
        type_hierarchy: Dict with type hierarchy.
        max_depth: Maximum depth of the type hierarchy.

    Returns:
        List with gain values corresponding to each item in predicted_types.
    """
    gains = []
    expanded_gold_types = get_expanded_types(gold_types, type_hierarchy)
    for predicted_type in predicted_types:
        if predicted_type in expanded_gold_types:
            # Since not all gold types may lie on the same branch, we take the
            # closest gold type for determining distance.
            min_distance = math.inf
            for gold_type in gold_types:
                min_distance = min(get_type_distance(predicted_type, gold_type,
                                                     type_hierarchy),
                                   min_distance)
            gains.append(1 - min_distance / max_depth)
        else:
            gains.append(0)
    return gains

def evaluate(result):
    """
    Evaluate the resulting dictionary, compute accuracy, strict and fuzzy ndcg_5, ndcg_10 where
    ndcg_5 and ndcg_10 is computed using lenient NDCG@k with a Linear decay.
    
    Arguments:
        result: A dictionary with queryIDs: List of retrieved types from the top 10 docs, 
        and a bool indicating if there was a perfect match.
    """
    accuracy = []
    strict_ndcg_5, strict_ndcg_10 = [], []
    for qId, obj in train_queries.items():
        if qId not in result:
            continue

        qTypes = obj['type'].split(' ')
        if len(qTypes) == 0:
            continue

        predicted_type = result[qId]
        predicted_type_strict = [(1 if (t in obj['type']) else 0) for t in predicted_type]        
        exact_match = max(predicted_type_strict + [0])
        
        # Filters obj types to most specific ones in the hierarchy.
        obj_types = get_most_specific_types(qTypes, type_hierarchy)
        gains = compute_type_gains(predicted_type, obj_types, type_hierarchy, max_depth)
        ideal_gains = sorted(gains, reverse=True)

        accuracy.append(exact_match)
        
        strict_ndcg_5.append(ndcg(predicted_type_strict, sorted(predicted_type_strict, reverse=True), k=5))
        strict_ndcg_10.append(ndcg(predicted_type_strict, sorted(predicted_type_strict, reverse=True), k=10))
        
    print('Evaluation results (based on {} questions):'.format(len(accuracy)))
    print('-------------------')
    
    print('Exact Type Prediction')
    print('  Accuracy: {:5.3f}'.format(sum(accuracy) / len(accuracy)))
    
    print('Strict Type ranking')
    print('  NDCG@5:  {:5.3f}'.format(sum(strict_ndcg_5) / len(strict_ndcg_5)))
    print('  NDCG@10: {:5.3f}'.format(sum(strict_ndcg_10) / len(strict_ndcg_10)))

In [52]:
RESULT_PATH = os.path.join(DATASETS_PATH, 'results')
isExist = os.path.exists(RESULT_PATH)
if not isExist:
   os.makedirs(RESULT_PATH)

def write_result_to_file(res, file):
    
    with open(os.path.join(RESULT_PATH, f'{file}.csv'), 'w') as f:
        for qId, obj in res.items():
            f.write('{},{}\n'.format(qId, ' '.join(obj)))

def read_result_from_file(file):
    result = {}
    with open(os.path.join(RESULT_PATH, f'{file}.csv'), 'r') as f:
        for line in f:
            line = line.strip().split(',')
            if len(line) != 2:
                continue
            result[line[0]] = [v for v in line[-1].split(' ') if len(v) > 0]
    return result

In [53]:
write_result_to_file(train_retrieval_results, 'BM25_baseline')
res = read_result_from_file("BM25_baseline")

In [54]:
evaluate(train_retrieval_results)

Evaluation results (based on 9573 questions):
-------------------
Exact Type Prediction
  Accuracy: 0.361
Strict Type ranking
  NDCG@5:  0.177
  NDCG@10: 0.233
